<a href="https://colab.research.google.com/github/brianellis1997/Music_Generation/blob/main/Model_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Demo
Here we will demo the model (XGBoost) and how it can give popularity predictions for our generative music.

In [1]:
!git clone https://github.com/brianellis1997/Music_Generation.git # Clone our repository

Cloning into 'Music_Generation'...
remote: Enumerating objects: 192, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 192 (delta 95), reused 37 (delta 7), pack-reused 0
Receiving objects: 100% (192/192), 23.41 MiB | 10.85 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from music21 import *

# Load MIDI file
midi_file_path = '/content/drive/MyDrive/DS340/Generated_2nd_stage.mid'
midi_stream = converter.parse(midi_file_path)

In [4]:
# Extraction class
class Extraction:
    def __init__(self, midi_stream):
        self.midi_stream = midi_stream

    def key_signature_extract(self):
        key_signatures = self.midi_stream.recurse().getElementsByClass(key.KeySignature)
        if len(key_signatures) > 0:
            key_signature = key_signatures[0]
            key_name = key_signature.asKey().tonic.name  # Get the key name (e.g., 'C', 'G#')
        else:
            key_analysis = self.midi_stream.analyze('key')
            key_name = key_analysis.tonic.name  # Get the key name from the analysis

        # Define a mapping from key names to Spotify's numbers
        key_to_number = {
            'C': 0, 'C#': 1, 'D-': 1, 'D': 2, 'D#': 3, 'E-': 3, 'E': 4,
            'F': 5, 'F#': 6, 'G-': 6, 'G': 7, 'G#': 8, 'A-': 8,
            'A': 9, 'A#': 10, 'B-': 10, 'B': 11
        }

        # Account for both sharp and flat representations
        if key_name in key_to_number:
            return key_to_number[key_name]
        else:
            # Handle the case where the key might be represented differently (e.g., flats)
            # music21 might represent some keys differently, e.g., 'F#' could also be 'G-' (G flat)
            # This is a placeholder for handling such cases
            print("Key name not found in mapping:", key_name)
            return None


    def tempo_extract(self):
        tempos = []
        for event in self.midi_stream.flatten():
            if 'MetronomeMark' in event.classes:
                tempos.append(int(event.number))
        if tempos:
            return sum(tempos) / len(tempos)
        else:
            print("Tempo information not found in the MIDI file.")
            return None

    def duration_extract(self):
        quarter_lengths = self.midi_stream.duration.quarterLength
        tempo_value = self.tempo_extract()
        if tempo_value:  # Ensure tempo_value is not None
            duration_min = quarter_lengths / tempo_value
            duration_ms = duration_min * 60000  # Convert minutes to milliseconds
            return duration_ms
        else:
            return None

    def valence_extract(self):
        major_chords_count = 0
        minor_chords_count = 0
        chords = self.midi_stream.chordify()
        for chord in chords.recurse().getElementsByClass('Chord'):
            if chord.isMajorTriad():
                major_chords_count += 1
            elif chord.isMinorTriad():
                minor_chords_count += 1
        if major_chords_count > 0:
            return major_chords_count / (major_chords_count + minor_chords_count) if minor_chords_count > 0 else 1
        return 0

    def mode_extract(self):
        key = self.key_signature_extract()
        key_str = str(key)
        if 'major' in key_str:
            return 1  # Major mode
        else:
            return 0  # Minor or other modes

    def extract_danceability(self):
        # Assuming you've defined tempo_weight, mode_weight, and valence_weight previously
        tempo = self.tempo_extract()
        mode = self.mode_extract()
        valence = self.valence_extract()
        tempo_weight = 0.4
        mode_weight = 0.3
        valence_weight = 0.3
        normalized_tempo = min(max((tempo - 60) / (180 - 60), 0), 1) if tempo else 0
        danceability_score = (normalized_tempo * tempo_weight) + (mode * mode_weight) + (valence * valence_weight)
        return danceability_score

    def estimate_energy(self):
        avg_tempo = self.tempo_extract()
        notes_and_chords = self.midi_stream.recurse().notes
        total_duration = self.midi_stream.duration.quarterLength
        note_density = len(notes_and_chords) / total_duration if total_duration > 0 else 0
        velocities = [n.volume.velocityScalar for n in notes_and_chords if n.volume.velocityScalar is not None]
        avg_velocity = sum(velocities) / len(velocities) if velocities else 0.5
        energy_score = (avg_tempo / 120) + (note_density * 2) + (avg_velocity * 2)
        return min(energy_score / 10, 1.0)

    def loudness_extract(self):
        # Assuming you want to calculate the average velocity for the normalization process
        notes = self.midi_stream.recurse().notes
        velocities = [note.volume.velocity for note in notes if note.volume.velocity is not None]

        if velocities:
            avg_velocity = sum(velocities) / len(velocities)
        else:
            avg_velocity = 0  # Use a sensible default if no notes are found

        # Call the static method correctly using the class name
        avg_loudness = self.normalize_loudness(avg_velocity)
        return avg_loudness

    @staticmethod
    def normalize_loudness(velocity, min_loudness=-60, max_loudness=3.855):
        # Normalize MIDI velocity from 0-127 to 0-1
        normalized_velocity = velocity / 127

        # Scale to target loudness range
        scaled_loudness = (normalized_velocity * (max_loudness - min_loudness)) + min_loudness

        return scaled_loudness

In [7]:
# Initialize the Extraction class with the midi_stream
extraction = Extraction(midi_stream)

# Extract features
generated_key = extraction.key_signature_extract()
generated_tempo = extraction.tempo_extract()
generated_duration = extraction.duration_extract()
generated_valence = extraction.valence_extract()
generated_mode = extraction.mode_extract()
generated_danceability = extraction.extract_danceability()
generated_energy = extraction.estimate_energy()
generated_loudness = extraction.loudness_extract()

# Print extracted features
print("Key:", generated_key)
print("Tempo:", generated_tempo)
print("Duration:", generated_duration)
print("Valence:", generated_valence)
print("Mode:", generated_mode)
print("Danceability:", generated_danceability)
print("Energy:", generated_energy)
print("Loudness:", generated_loudness)

Key: 1
Tempo: 120.51982378854626
Duration: 246930.33116455883
Valence: 0.7735849056603774
Mode: 0
Danceability: 0.4338082176599341
Energy: 0.7516624991813022
Loudness: -29.48237006159789


In [5]:
# Load model
import xgboost as xgb

In [6]:
# Initialize a model instance
loaded_model = xgb.XGBRegressor()

# Load the saved model
loaded_model.load_model("/content/drive/MyDrive/DS340/best_xgb_model.json")  # Adjust path if necessary

In [9]:
import numpy as np
import joblib  # For loading the scaler

# Assuming 'Extraction' is your class for feature extraction
class PredictionPipeline:
    def __init__(self, model_path, scaler_path):
        # Load the XGBoost model
        self.model = xgb.XGBRegressor()
        self.model.load_model(model_path)

        # Load the scaler
        self.scaler = joblib.load(scaler_path)

        # Initialize the Extraction object (placeholder, need a midi_stream)
        self.extraction = None

    def extract_features(self, midi_path):
        # Load the MIDI file
        midi_stream = converter.parse(midi_path)
        self.extraction = Extraction(midi_stream)

        # Extract features
        features = np.array([
            self.extraction.valence_extract(),
            self.extraction.extract_danceability(),
            self.extraction.duration_extract(),
            self.extraction.estimate_energy(),
            self.extraction.key_signature_extract(),
            self.extraction.loudness_extract(),
            self.extraction.mode_extract(),
            self.extraction.tempo_extract()
        ]).reshape(1, -1)  # Reshape for a single sample

        return features

    def predict(self, midi_path):
        # Extract features
        features = self.extract_features(midi_path)

        # Scale features
        features_scaled = self.scaler.transform(features)

        # Make prediction
        prediction = self.model.predict(features_scaled)

        return prediction

# Example usage
pipeline = PredictionPipeline('/content/drive/MyDrive/DS340/best_xgb_model.json', '/content/drive/MyDrive/DS340/scaler.joblib')
prediction = pipeline.predict('/content/drive/MyDrive/DS340/Generated_2nd_stage.mid')
print("Prediction:", prediction)


Prediction: [25.183252]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
